In [13]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels.formula.api as smf
from datetime import datetime
import pymssql
from sqlalchemy import create_engine



### Input and output

In [14]:
input_file = 'files/TransactionHistory_3_23_2023.csv'
output_dir = "cqa_figures"

### Process transactions file

In [15]:
transact = pd.read_csv(input_file,parse_dates=['CreateDate'])
transact = transact.rename(columns={"Symbol": "ticker", "CreateDate": "date"})
transact['Price'] = transact.Price.apply(lambda x: x[1:]).astype(float)
transact['Quantity'] = transact.Quantity.astype(int)
transact['date'] = transact.date.dt.date.astype(str)
transact = transact.sort_values(by=["ticker", "date",'TransactionType'])
transact = transact.drop_duplicates(subset=['ticker','Quantity','Price','date'],keep='first')  # because short proceeds records show up twice (I think)
transact.Amount = transact.Amount.map(
    lambda x: float(x[1:].replace(",", "")) if x[0] != "(" else -float(x[2:-1].replace(",", ""))
)    

cashflow = transact.groupby("date").Amount.sum()
cash = 1000000 + cashflow.cumsum()

trades = transact[transact.TransactionType != 'Dividends']
trades = trades.groupby(["ticker", "date"]).Quantity.sum()

positions = trades.groupby("ticker").cumsum()       # sum up trades over time to get positions

### Read closing prices from Yahoo

In [16]:
prices = yf.download(list(transact.ticker.unique()), start="2022-11-04")

prices = prices.stack()
prices.index.names = ["date", "ticker"]
prices = prices.reset_index()[["date", "ticker", "Close"]]
prices.date = prices.date.dt.date.astype(str)
prices = prices.set_index(["ticker", "date"])

[*********************100%***********************]  96 of 96 completed

2 Failed downloads:
- FFHL: No timezone found, symbol may be delisted
- SPNE: No timezone found, symbol may be delisted


### Combine transactions with prices

In [17]:
df = prices.merge(positions, left_index=True, right_index=True, how="left")
indx = pd.MultiIndex.from_product(
    (df.index.unique("ticker"), df.index.unique("date"))
)
df = df.reindex(indx)                                       # expand dataframe to all (ticker, date) pairs

df["Price"] = transact.set_index(["ticker", "date"]).Price  # use transaction prices when available
df["Price"] = df.Price.fillna(df.Close)                     # use closing prices on non-transaction dates
df["Price"] = df.groupby("ticker").Price.ffill()            # filling forward for delisted stocks

df["Quantity"] = df.groupby("ticker").Quantity.ffill()      # fill quantities until next transaction date
df["Value"] = df.Quantity * df.Price

### Compute portfolio value and daily returns

In [18]:
portfolio = df.groupby("date").Value.sum()
portfolio = pd.DataFrame(portfolio).merge(cash, on="date", how="left")
portfolio.columns = ["stocks", "cash"]
portfolio["cash"] = portfolio.cash.ffill()
equity = portfolio.stocks + portfolio.cash
ret = equity.pct_change()

### Sharpe ratio, etc.

In [19]:
print(f"annualized average return = {252*ret.mean(): .1%}")
print(f"annualized std dev is {np.sqrt(252)*ret.std(): .1%}")
print(f"annualized Sharpe ratio is {np.sqrt(252)*ret.mean()/ret.std(): .1%}")

annualized average return =  40.5%
annualized std dev is  55.2%
annualized Sharpe ratio is  73.3%


### Plot cumulative return

In [20]:
cum_ret = equity/1000000
trace1 = go.Scatter(
    x = cum_ret.index.to_list(),
    y = cum_ret-1,
    mode="lines",
    hovertemplate="%{x}<br>cumulative ret = %{y:.2%}<extra></extra>"
)

fig = go.Figure(trace1)

fig.update_yaxes(
    title_text="<b>Cumulative Return</b>", 
    tickformat=".0%", 
)

fig.update_layout(
    template="plotly_white",
    font_size=16,
    showlegend=False,
)

fig.write_image(f"{output_dir}/cumret.jpeg")
with open(f"{output_dir}/cumret.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

### Plot drawdowns

In [21]:


cum_max = cum_ret.expanding().max()
drawdown = cum_ret/cum_max - 1

trace2 = go.Scatter(
    x = drawdown.index.to_list(),
    y = drawdown,
    mode="lines",
   hovertemplate="%{x}<br>drawdown = %{y:.2%}<extra></extra>"
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

plotly_template = pio.templates["plotly_white"]
colors = plotly_template.layout.colorway

fig.update_yaxes(
    title_text="<b>Cumulative Return</b>", 
    title_font=dict(color=colors[0]), 
    color=colors[0], 
    tickformat=".0%", 
    secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Drawdown</b>", 
    title_font=dict(color=colors[1]), 
    color=colors[1], 
    tickformat=".0%", 
    secondary_y=True
)

fig.update_layout(
    template="plotly_white",
    font_size=16,
    showlegend=False,
)

fig.write_image(f"{output_dir}/drawdown.jpeg")
with open(f"{output_dir}/drawdown.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

### Compare to S&P 500 (SPY)

In [22]:
spy = yf.download("SPY", start="2022-11-03")["Adj Close"].pct_change().dropna()
spy.index = [datetime.strftime(x, "%Y-%m-%d") for x in spy.index]

rets = pd.concat((ret, spy), axis=1)
rets.columns = ["ret", "spy"]

result = smf.ols("ret~spy", rets).fit()
alpha = result.params[0]
beta = result.params[1]
info_ratio = alpha / np.sqrt(result.mse_resid)

spy = beta*spy
active = ret - spy

trace1 = go.Scatter(
    x = ret.index.to_list(),
    y = (1+ret).cumprod()-1,
    name = "Total return",
    hovertemplate="%{x}<br>total=%{y:.2%}<extra></extra>"
)

trace2 = go.Scatter(
    x = active.index.to_list(),
    y = (1+active).cumprod()-1,
    name = "Active part of return",
    hovertemplate="%{x}<br>active=%{y:.2%}<extra></extra>"
)
trace3 = go.Scatter(
    x = spy.index.to_list(),
    y = (1+spy).cumprod()-1,
    name = "S&P 500 part of return",
    hovertemplate="%{x}<br>spy=%{y:.2%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Cumulative Return",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.write_image(f"{output_dir}/spy.jpeg")
with open(f"{output_dir}/spy.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

[*********************100%***********************]  1 of 1 completed


In [23]:
print(f"annualized alpha with respect to SPY = {252*result.params[0]:.1%}")
print(f"beta with respect to SPY = {result.params[1]:.2f}")
print(f"annualized info ratio with respect to SPY = {np.sqrt(252)* alpha / np.sqrt(result.mse_resid):.1%}")


annualized alpha with respect to SPY = 35.4%
beta with respect to SPY = 0.30
annualized info ratio with respect to SPY = 64.2%


### Compare to Russell 2000 (IWM)

In [24]:
spy = yf.download("IWM", start="2022-11-03")["Adj Close"].pct_change().dropna()
spy.index = [datetime.strftime(x, "%Y-%m-%d") for x in spy.index]

rets = pd.concat((ret, spy), axis=1)
rets.columns = ["ret", "spy"]

result = smf.ols("ret~spy", rets).fit()
alpha = result.params[0]
beta = result.params[1]
info_ratio = alpha / np.sqrt(result.mse_resid)

spy = beta*spy
active = ret - spy

trace1 = go.Scatter(
    x = ret.index.to_list(),
    y = (1+ret).cumprod()-1,
    name = "Total return",
    hovertemplate="%{x}<br>total=%{y:.2%}<extra></extra>"
)

trace2 = go.Scatter(
    x = active.index.to_list(),
    y = (1+active).cumprod()-1,
    name = "Active part of return",
    hovertemplate="%{x}<br>active=%{y:.2%}<extra></extra>"
)
trace3 = go.Scatter(
    x = spy.index.to_list(),
    y = (1+spy).cumprod()-1,
    name = "Russell 2000 part of return",
    hovertemplate="%{x}<br>iwm=%{y:.2%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Cumulative Return",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.write_image(f"{output_dir}/russell.jpeg")
with open(f"{output_dir}/russell.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

[*********************100%***********************]  1 of 1 completed


In [25]:
print(f"annualized alpha with respect to IWM = {252*result.params[0]:.1%}")
print(f"beta with respect to IWM = {result.params[1]:.2f}")
print(f"annualized info ratio with respect to IWM = {np.sqrt(252)* alpha / np.sqrt(result.mse_resid):.1%}")

annualized alpha with respect to IWM = 41.2%
beta with respect to IWM = 0.25
annualized info ratio with respect to IWM = 74.6%


### Fama-French attribution analysis

In [26]:
ff = pdr("F-F_Research_Data_5_Factors_2x3_daily", "famafrench", start="2022-11-04")[0]/100
ff = ff.rename(columns={"Mkt-RF":  "Mkt_RF"})
ff.index = ff.index.astype(str)
ff["xret"] = ret - ff.RF


In [27]:
result = smf.ols("xret~Mkt_RF+SMB+HML+CMA+RMW", data=ff.dropna()).fit()
print(result.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0003      0.004      0.078      0.938      -0.008       0.009
Mkt_RF        -0.1047      0.481     -0.218      0.829      -1.070       0.861
SMB            0.5063      1.110      0.456      0.650      -1.722       2.734
HML           -1.9136      0.982     -1.949      0.057      -3.884       0.056
CMA            0.6363      1.436      0.443      0.659      -2.245       3.517
RMW            0.9491      1.104      0.860      0.394      -1.266       3.164


In [28]:
info = np.sqrt(252)*result.params["Intercept"]/np.sqrt(result.mse_resid)
print(f"annualized FF information ratio is {info:.1%}")

annualized FF information ratio is 16.9%


In [29]:
betas = result.params[1:]
mkt = betas[0]*ff.Mkt_RF 
smb = betas[1]*ff.SMB
hml = betas[2]*ff.HML
cma = betas[3]*ff.CMA
rmw = betas[4]*ff.RMW
active = ff.xret - mkt - smb - hml - cma - rmw

trace1 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+ff.xret).cumprod()-1,
    name = "Total",
    hovertemplate="%{x}<br>total=%{y:.1%}<extra></extra>"
)
trace2 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+mkt).cumprod()-1,
    name = "Market",
    hovertemplate="%{x}<br>market=%{y:.1%}<extra></extra>"
)
trace3 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+smb).cumprod()-1,
    name = "SMB",
    hovertemplate="%{x}<br>smb=%{y:.1%}<extra></extra>"
)
trace4 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+hml).cumprod()-1,
    name = "HML",
    hovertemplate="%{x}<br>hml=%{y:.1%}<extra></extra>"
)
trace5 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+cma).cumprod()-1,
    name = "CMA",
    hovertemplate="%{x}<br>cma=%{y:.1%}<extra></extra>"
)
trace6 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+rmw).cumprod()-1,
    name = "RMW",
    hovertemplate="%{x}<br>rmw=%{y:.1%}<extra></extra>"
)
trace7 = go.Scatter(
    x = ff.index.to_list(),
    y = (1+active).cumprod()-1,
    name = "Active",
    hovertemplate="%{x}<br>active=%{y:.1%}<extra></extra>"
)
fig = go.Figure()
for trace in [trace1, trace2, trace3, trace4, trace5, trace6, trace7]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Cumulative Return",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)
fig.write_image(f"{output_dir}/attribution.jpeg")
with open(f"{output_dir}/attribution.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

### Get industries and some stock characteristics

In [30]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 

conn = create_engine(string).connect()

In [31]:
today = pd.read_sql("select ticker, famaindustry, agr, bm, marketcap, operprof, mom12m, retvol from today", conn)
complete = df.reset_index().merge(today, on="ticker", how="outer")

oneday = complete[complete.date=="2023-03-01"].copy()
longs = oneday[oneday.Quantity>0]
shorts = oneday[oneday.Quantity<0]
shorts["Value"] = - shorts.Value


C:\Users\keb7\AppData\Local\Temp\ipykernel_106440\471500484.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Long industries

In [32]:
fig = px.pie(longs.dropna(subset=["Value", "famaindustry"]), values="Value", names="famaindustry")
fig.write_image(f"{output_dir}/long_industries.jpeg")
with open(f"{output_dir}/long_industries.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

### Short industries

In [33]:
fig = px.pie(shorts.dropna(subset=["Value", "famaindustry"]), values="Value", names="famaindustry")
fig.write_image(f"{output_dir}/short_industries.jpeg")
with open(f"{output_dir}/short_industries.html", "w", encoding="utf-8") as f:
    f.write(fig.to_html()) 
fig.show()

### Net exposures: size and characteristics

In [34]:
for char in "agr bm marketcap operprof mom12m retvol".split():
    oneday[char+"_group"] = pd.qcut(complete[char], 5, labels=range(1, 6))

agr = oneday.dropna(subset=["marketcap", "agr", "Value"]).groupby(["marketcap_group", "agr_group"]).Value.sum().unstack()
bm = oneday.dropna(subset=["marketcap", "bm", "Value"]).groupby(["marketcap_group", "bm_group"]).Value.sum().unstack()
operprof = oneday.dropna(subset=["marketcap", "operprof", "Value"]).groupby(["marketcap_group", "operprof_group"]).Value.sum().unstack()
mom12m = oneday.dropna(subset=["marketcap", "mom12m", "Value"]).groupby(["marketcap_group", "mom12m_group"]).Value.sum().unstack()
retvol = oneday.dropna(subset=["marketcap", "retvol", "Value"]).groupby(["marketcap_group", "retvol_group"]).Value.sum().unstack()

In [35]:
chars = "agr bm operprof mom12m retvol".split()
data = [agr, bm, operprof, mom12m, retvol]

for char, d in zip(chars, data):
    trace = go.Heatmap(
        x=d.columns.to_list(),
        y=d.index.to_list(),
        z=d,
        text = [[char]*5]*5,
        colorscale='Viridis',
        texttemplate="%{z:,.0f}",
        hovertemplate="size quintile = %{y}<br>%{text} quintile = %{x}<br>net investment = $%{z:,.0f}<extra></extra>"
    )
    fig = go.Figure(trace)
    fig.update_layout(
        xaxis_title=f"{char} Quintile (low to high)",
        yaxis_title="Size Quintile (low to high)",
        template="plotly_white",
        font_size=16
)
    fig.write_image(f"{output_dir}/{char}.jpeg")
    with open(f"{output_dir}/{char}.html", "w", encoding="utf-8") as f:
        f.write(fig.to_html()) 
    fig.show()